- Environment Setup

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import RidgeClassifier, RidgeClassifierCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import export_graphviz

import graphviz
from graphviz import Graph

import wrangle

from wrangle import get_311_data, clean_311

import model

from model import split_separate_scale, model_df

pd.set_option("display.max_rows", None, "display.max_columns", None)

# Acquire and Prep

In [2]:
# Acquire and prep data
df = model_df()

In [3]:
df.head()

,open_date,dept,call_reason,source_id,council_district,resolution_days_due,level_of_delay,district_1,district_2,district_3,district_4,district_5,district_6,district_7,district_8,district_9,district_10,animal_care_services,code_enforcement_services,customer_services,development_services,metro_health,parks_and_rec,solid_waste_management,trans_and_cap_improvements,unknown_dept,buildings,business,cleanup,code,customer_service,field,land,license,misc,storm,streets,trades,traffic,waste,web_portal,311_mobile_app,constituent_call,internal_services_requests,open_month,open_year,open_week
551,2017-01-27,Customer Service,customer_service,Constituent Call,2,8,Very Late Response,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,2017,4
552,2017-01-27,Customer Service,customer_service,Constituent Call,4,8,Very Late Response,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,2017,4
553,2017-02-03,Customer Service,customer_service,Constituent Call,2,8,Very Late Response,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,2,2017,5
554,2017-02-03,Customer Service,customer_service,Constituent Call,1,8,Very Late Response,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,2,2017,5
555,2017-02-28,Customer Service,customer_service,Constituent Call,6,8,Very Late Response,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,2,2017,9


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356804 entries, 551 to 399979
Data columns (total 47 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   open_date                   356804 non-null  datetime64[ns]
 1   dept                        356804 non-null  object        
 2   call_reason                 356804 non-null  object        
 3   source_id                   356804 non-null  object        
 4   council_district            356804 non-null  int64         
 5   resolution_days_due         356804 non-null  int64         
 6   level_of_delay              356804 non-null  category      
 7   district_1                  356804 non-null  uint8         
 8   district_2                  356804 non-null  uint8         
 9   district_3                  356804 non-null  uint8         
 10  district_4                  356804 non-null  uint8         
 11  district_5                  356804 no

In [5]:
df.describe()

,council_district,resolution_days_due,district_1,district_2,district_3,district_4,district_5,district_6,district_7,district_8,district_9,district_10,animal_care_services,code_enforcement_services,customer_services,development_services,metro_health,parks_and_rec,solid_waste_management,trans_and_cap_improvements,unknown_dept,buildings,business,cleanup,code,customer_service,field,land,license,misc,storm,streets,trades,traffic,waste,web_portal,311_mobile_app,constituent_call,internal_services_requests,open_month,open_year,open_week
count,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000,356804.000000
mean,4.802188,32.784952,0.129236,0.132218,0.128648,0.097440,0.140629,0.085532,0.094158,0.054646,0.054728,0.082765,0.160419,0.004762,0.007803,0.262183,0.006603,0.000006,0.473114,0.030731,0.054380,0.000499,0.003756,0.000177,0.264523,0.015353,0.160606,0.004863,0.002214,0.057398,0.009591,0.028301,0.025190,0.017007,0.410522,0.108858,0.084085,0.001256,0.805801,6.345714,2020.316185,26.108614
std,2.780184,45.078669,0.335462,0.338728,0.334810,0.296556,0.347639,0.279671,0.292049,0.227289,0.227448,0.275528,0.366994,0.068841,0.087987,0.439822,0.080991,0.002368,0.499277,0.172589,0.226766,0.022330,0.061168,0.013287,0.441080,0.122952,0.367168,0.069563,0.047002,0.232603,0.097462,0.165832,0.156703,0.129296,0.491929,0.311462,0.277516,0.035412,0.395583,3.227659,0.516766,14.388064
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2017.000000,1.000000
25%,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,4.000000,2020.000000,14.000000
50%,5.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.000000,2020.000000,26.000000
75%,7.000000,65.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,9.000000,2021.000000,38.000000
max,10.000000,934.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,12.000000,2021.000000,53.000000


In [6]:
df.isnull().sum()

open_date                     0
dept                          0
call_reason                   0
source_id                     0
council_district              0
resolution_days_due           0
level_of_delay                0
district_1                    0
district_2                    0
district_3                    0
district_4                    0
district_5                    0
district_6                    0
district_7                    0
district_8                    0
district_9                    0
district_10                   0
animal_care_services          0
code_enforcement_services     0
customer_services             0
development_services          0
metro_health                  0
parks_and_rec                 0
solid_waste_management        0
trans_and_cap_improvements    0
unknown_dept                  0
buildings                     0
business                      0
cleanup                       0
code                          0
customer_service              0
field   

# Modeling

## Split Data 

In [7]:
# Run split function
train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test, train_scaled, validate_scaled, test_scaled = split_separate_scale(df)

## Establish Baseline

In [8]:
# look at values of target variable
y_train.value_counts()

Very Early Response    114280
Early Response          36971
Late Response           33599
On Time Response        11753
Very Late Response       3207
Name: level_of_delay, dtype: int64

In [9]:
# set up as dataframes
y_train = pd.DataFrame(dict(actual=y_train))
y_validate = pd.DataFrame(dict(actual=y_validate))
y_test = pd.DataFrame(dict(actual=y_test))

In [10]:
# 'Extremely Early Response' is by far the most frequent value so that will be our baseline
y_train['baseline'] = 'Very Early Response'

In [11]:
# calculate accuracy of baseline
baseline_accuracy = accuracy_score(y_train.actual, y_train.baseline)
print(' Baseline Accuracy: {:.2%}'.format(accuracy_score(y_train.actual, y_train.baseline)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(y_train.baseline, y_train.actual))
print('---')
print(classification_report(y_train.actual, y_train.baseline))

 Baseline Accuracy: 57.19%
---
Confusion Matrix
actual               Very Early Response  Early Response  On Time Response  \
baseline                                                                     
Very Early Response               114280           36971             11753   

actual               Late Response  Very Late Response  
baseline                                                
Very Early Response          33599                3207  
---
                     precision    recall  f1-score   support

     Early Response       0.00      0.00      0.00     36971
      Late Response       0.00      0.00      0.00     33599
   On Time Response       0.00      0.00      0.00     11753
Very Early Response       0.57      1.00      0.73    114280
 Very Late Response       0.00      0.00      0.00      3207

           accuracy                           0.57    199810
          macro avg       0.11      0.20      0.15    199810
       weighted avg       0.33      0.57      0.42  

### Takeaways

- Baseline accuracy is 57.19%, which means we can assume approx 1/2 of cases are resolved very early

- Our goal will be to outperform this baseline

## Logistic Regression - Default Parameters / All Features

- The first model we looked at was logistic regression

- Various iterations were made, but the logistic regression model that worked the best turned out to be a model using default parameters and all features

In [12]:
# make logistic regression object
log = LogisticRegression(random_state=123)
# fit logistic regression object to data
log.fit(train_scaled, y_train.actual)
# calculate probability
log_pred_proba = log.predict_proba(train_scaled)
# make predictions
y_train['log_pred'] = log.predict(train_scaled)
y_validate['log_pred'] = log.predict(validate_scaled)
y_test['log_pred'] = log.predict(test_scaled)

In [13]:
# calculate accuracy of log on train
log_accuracy_train = accuracy_score(y_train.actual, y_train.log_pred)                                     
print(' log Accuracy: {:.5%}'.format(accuracy_score(y_train.actual, y_train.log_pred)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(y_train.log_pred, y_train.actual))
print('---')
print(classification_report(y_train.actual, y_train.log_pred))

 log Accuracy: 59.30184%
---
Confusion Matrix
actual               Very Early Response  Early Response  On Time Response  \
log_pred                                                                     
Early Response                       129              92                43   
Late Response                       4763            2700               553   
Very Early Response               109387           34174             11157   
Very Late Response                     1               5                 0   

actual               Late Response  Very Late Response  
log_pred                                                
Early Response                  48                  13  
Late Response                 7319                 958  
Very Early Response          26058                 543  
Very Late Response             174                1693  
---
                     precision    recall  f1-score   support

     Early Response       0.28      0.00      0.00     36971
      Late Respo

In [14]:
# calculate accuracy of log on validate
log_accuracy_validate = accuracy_score(y_validate.actual, y_validate.log_pred)
print(' Log Accuracy: {:.5%}'.format(accuracy_score(y_validate.actual, y_validate.log_pred)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(y_validate.log_pred, y_validate.actual))
print('---')
print(classification_report(y_validate.actual, y_validate.log_pred))

 Log Accuracy: 59.34394%
---
Confusion Matrix
actual               Very Early Response  Early Response  On Time Response  \
log_pred                                                                     
Early Response                        46              37                22   
Late Response                       2060            1161               238   
Very Early Response                46868           14645              4777   
Very Late Response                     4               2                 0   

actual               Late Response  Very Late Response  
log_pred                                                
Early Response                  21                  11  
Late Response                 3155                 380  
Very Early Response          11152                 225  
Very Late Response              71                 758  
---
                     precision    recall  f1-score   support

     Early Response       0.27      0.00      0.00     15845
      Late Respo

### Takeaways

- This model only outperforms the baseline by about 2%

- an improvement, but not by much

## Decision Tree - Deeper Max Depth - Selected Features

- Next we looked at the Decision Tree model

- The best iteration of this model was found by doubling the max depth parameter as well as only incorporating features we found to be significant

In [15]:
# make seperate df by removing incompatible features 
dt_train = X_train.drop(columns = ['dept', 'call_reason', 
        'source_id', 'council_district', 'open_date'])
dt_validate = X_validate.drop(columns = ['dept', 'call_reason', 
        'source_id', 'council_district', 'open_date'])
dt_test = X_test.drop(columns = ['dept', 'call_reason', 
        'source_id', 'council_district', 'open_date'])

In [16]:
#make the thing
clf = DecisionTreeClassifier(max_depth=6, random_state=123)
#fit the thing
clf = clf.fit(dt_train, y_train.actual)
#predicitons
y_train['dt_pred'] = clf.predict(dt_train)
y_validate['dt_pred'] = clf.predict(dt_validate)
y_test['dt_pred'] = clf.predict(dt_test)
# make visualization of Decision Tree
dot_data = export_graphviz(clf, feature_names= dt_train.columns, class_names=clf.classes_, rounded=True, filled=True, out_file=None)
graph = graphviz.Source(dot_data) 

graph.render('311_decision_tree', view=True)

'311_decision_tree.pdf'

In [17]:
# calculate accuracy of dt on train
dt_accuracy_train = accuracy_score(y_train.actual, y_train.dt_pred)                                     
print(' DT Accuracy: {:.5%}'.format(accuracy_score(y_train.actual, y_train.dt_pred)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(y_train.dt_pred, y_train.actual))
print('---')
print(classification_report(y_train.actual, y_train.dt_pred))

 DT Accuracy: 67.96507%
---
Confusion Matrix
actual               Very Early Response  Early Response  On Time Response  \
dt_pred                                                                      
Early Response                      1242            9699              1833   
Late Response                       1055            1730               552   
On Time Response                       0               0                 1   
Very Early Response               111983           25542              9367   
Very Late Response                     0               0                 0   

actual               Late Response  Very Late Response  
dt_pred                                                 
Early Response                3039                 149  
Late Response                11911                 581  
On Time Response                 0                   0  
Very Early Response          18646                 270  
Very Late Response               3                2207  
---
     

In [18]:
# calculate accuracy of dt on validate
dt_accuracy_validate = accuracy_score(y_validate.actual, y_validate.dt_pred)                                     
print(' DT Accuracy: {:.5%}'.format(accuracy_score(y_validate.actual, y_validate.dt_pred)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(y_validate.dt_pred, y_validate.actual))
print('---')
print(classification_report(y_validate.actual, y_validate.dt_pred))

 DT Accuracy: 68.01467%
---
Confusion Matrix
actual               Very Early Response  Early Response  On Time Response  \
dt_pred                                                                      
Early Response                       546            4197               821   
Late Response                        505             721               218   
Very Early Response                47927           10927              3998   
Very Late Response                     0               0                 0   

actual               Late Response  Very Late Response  
dt_pred                                                 
Early Response                1328                  53  
Late Response                 5143                 246  
Very Early Response           7927                  99  
Very Late Response               1                 976  
---
                     precision    recall  f1-score   support

     Early Response       0.60      0.26      0.37     15845
      Late Respon

### Takeaways

- This model improves on the baseline by about 10%

- Much better but can still be improved 

## K Nearest Neighbors - Uniform Weight / Chosen Features

- Next we looked at KNN Models

- The model that performed the best was a model using uniform weight and selecting features that were found to be significant

In [19]:
#Features I want to train on
features1 = ['council_district',
 'resolution_days_due',
 'district_1',
 'district_2',
 'district_3',
 'district_4',
 'district_5',
 'district_6',
 'district_7',
 'district_8',
 'district_9',
 'district_10',
 'animal_care_services',
 'code_enforcement_services',
 'customer_services',
 'development_services',
 'metro_health',
 'parks_and_rec',
 'solid_waste_management',
 'trans_and_cap_improvements',
 'unknown_dept',
 'buildings',
 'business',
 'cleanup',
 'code',
 'customer_service',
 'field',
 'land',
 'license',
 'misc',
 'storm',
 'streets',
 'trades',
 'traffic',
 'waste',
 'web_portal',
 '311_mobile_app',
 'constituent_call',
 'internal_services_requests']
#make model
#weights = ['uniform', 'density']
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')
#fit model
knn.fit(X_train[features1], y_train.actual)
#predicitons
y_train['knn_pred'] = knn.predict(X_train[features1])
y_validate['knn_pred'] = knn.predict(X_validate[features1])
y_test['knn_pred'] = knn.predict(X_test[features1])
#estimate probability
knn_pred_proba = knn.predict_proba(X_train[features1])

In [20]:
# calculate accuracy of knn on train
knn_accuracy_train = accuracy_score(y_train.actual, y_train.knn_pred)                                     
print(' KNN Accuracy: {:.5%}'.format(accuracy_score(y_train.actual, y_train.knn_pred)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(y_train.knn_pred, y_train.actual))
print('---')
print(classification_report(y_train.actual, y_train.knn_pred))

 KNN Accuracy: 60.23923%
---
Confusion Matrix
actual               Very Early Response  Early Response  On Time Response  \
knn_pred                                                                     
Early Response                     12708           14668              3326   
Late Response                       8675            4257              1732   
On Time Response                    2704            1641              1117   
Very Early Response                90112           16359              5555   
Very Late Response                    81              46                23   

actual               Late Response  Very Late Response  
knn_pred                                                
Early Response                6081                 412  
Late Response                12729                 543  
On Time Response               957                  59  
Very Early Response          13797                 455  
Very Late Response              35                1738  
---
    

In [21]:
# calculate accuracy of knn on validate
knn_accuracy_validate = accuracy_score(y_validate.actual, y_validate.knn_pred)                                     
print(' KNN Accuracy: {:.5%}'.format(accuracy_score(y_validate.actual, y_validate.knn_pred)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(y_validate.knn_pred, y_validate.actual))
print('---')
print(classification_report(y_validate.actual, y_validate.knn_pred))

 KNN Accuracy: 59.45138%
---
Confusion Matrix
actual               Very Early Response  Early Response  On Time Response  \
knn_pred                                                                     
Early Response                      5658            6092              1505   
Late Response                       3736            1861               691   
On Time Response                    1211             744               442   
Very Early Response                38324            7131              2389   
Very Late Response                    49              17                10   

actual               Late Response  Very Late Response  
knn_pred                                                
Early Response                2592                 180  
Late Response                 5286                 211  
On Time Response               422                  29  
Very Early Response           6077                 188  
Very Late Response              22                 766  
---
    

### Takeaways

- This model performs similarly compared to the baseline

- This model should not be used in future iterations

## Random Forest - Increased Max Depth & Min Samples / Best Features

- Next we moved on to Random Forest modeling

- The best version of this model we found was when we increased the max depth and min samples leaf paramaters and used feature engineering to determine the best features 

In [22]:
# make seperate df by removing incompatible features 
rf_train = X_train.drop(columns = ['dept', 'call_reason', 
        'source_id', 'council_district', 'open_date'])
rf_validate = X_validate.drop(columns = ['dept', 'call_reason', 
        'source_id', 'council_district', 'open_date'])
rf_test = X_test.drop(columns = ['dept', 'call_reason', 
        'source_id', 'council_district', 'open_date'])

# make list of features to use
features = ['resolution_days_due', 'code_enforcement_services','customer_services',
                                       'development_services','solid_waste_management',
                                       'open_week']

In [23]:
#make the thing
rf = RandomForestClassifier(max_depth = 10, min_samples_leaf = 3, random_state=123)
#fit the thing
rf = rf.fit(rf_train[features], y_train.actual)
#predicitons
y_train['rf_pred'] = rf.predict(rf_train[features])
y_validate['rf_pred'] = rf.predict(rf_validate[features])
y_test['rf_pred'] = rf.predict(rf_test[features])

In [24]:
# calculate accuracy of rf on train
rf_accuracy_train = accuracy_score(y_train.actual, y_train.rf_pred)                                     
print(' RF Accuracy: {:.5%}'.format(accuracy_score(y_train.actual, y_train.rf_pred)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(y_train.rf_pred, y_train.actual))
print('---')
print(classification_report(y_train.actual, y_train.rf_pred))

 RF Accuracy: 66.55923%
---
Confusion Matrix
actual               Very Early Response  Early Response  On Time Response  \
rf_pred                                                                      
Early Response                      1231            9516              1766   
Late Response                       1288            1647               327   
On Time Response                       1               5                16   
Very Early Response               111757           25803              9643   
Very Late Response                     3               0                 1   

actual               Late Response  Very Late Response  
rf_pred                                                 
Early Response                2752                 227  
Late Response                10129                 478  
On Time Response                 4                   0  
Very Early Response          20714                 928  
Very Late Response               0                1574  
---
     

In [25]:
# calculate accuracy of rf on validate
rf_accuracy_validate = accuracy_score(y_validate.actual, y_validate.rf_pred)                                     
print(' RF Accuracy: {:.5%}'.format(accuracy_score(y_validate.actual, y_validate.rf_pred)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(y_validate.rf_pred, y_validate.actual))
print('---')
print(classification_report(y_validate.actual, y_validate.rf_pred))

 RF Accuracy: 66.43934%
---
Confusion Matrix
actual               Very Early Response  Early Response  On Time Response  \
rf_pred                                                                      
Early Response                       540            4107               789   
Late Response                        604             720               152   
On Time Response                       0               6                 5   
Very Early Response                47833           11012              4091   
Very Late Response                     1               0                 0   

actual               Late Response  Very Late Response  
rf_pred                                                 
Early Response                1195                  96  
Late Response                 4260                 188  
On Time Response                 3                   0  
Very Early Response           8940                 401  
Very Late Response               1                 689  
---
     

### Takeaways

- This model appears to improve on the baseline by about 9%

## Ridge Classifier - Default Parameters / Selected Features 

- Next we looked into the Ridge Classifier model

- The best version of this model we found was using default parameters and our selected features 

In [26]:
#make the thing
rc_cv = RidgeClassifierCV()
#fit the thing
rc_cv = rc_cv.fit(train_scaled, y_train.actual)
#predicitons
y_train['rc_cv_pred'] = rc_cv.predict(train_scaled)
y_validate['rc_cv_pred'] = rc_cv.predict(validate_scaled)
y_test['rc_cv_pred'] = rc_cv.predict(test_scaled)

In [27]:
# calculate accuracy of rc_cv on train
rc_cv_accuracy_train = accuracy_score(y_train.actual, y_train.rc_cv_pred)                                     
print(' RC_CV Accuracy: {:.5%}'.format(accuracy_score(y_train.actual, y_train.rc_cv_pred)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(y_train.rc_cv_pred, y_train.actual))
print('---')
print(classification_report(y_train.actual, y_train.rc_cv_pred))

 RC_CV Accuracy: 58.98954%
---
Confusion Matrix
actual               Very Early Response  Early Response  On Time Response  \
rc_cv_pred                                                                   
Early Response                        59              34                22   
Late Response                       3002            1867               395   
Very Early Response               111219           35070             11336   
Very Late Response                     0               0                 0   

actual               Late Response  Very Late Response  
rc_cv_pred                                              
Early Response                  22                   2  
Late Response                 5074                 942  
Very Early Response          28503                 723  
Very Late Response               0                1540  
---
                     precision    recall  f1-score   support

     Early Response       0.24      0.00      0.00     36971
      Late Res

In [28]:
# calculate accuracy of rc_cv on validate
rc_cv_accuracy_validate = accuracy_score(y_validate.actual, y_validate.rc_cv_pred)                                     
print(' RC_CV Accuracy: {:.5%}'.format(accuracy_score(y_validate.actual, y_validate.rc_cv_pred)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(y_validate.rc_cv_pred, y_validate.actual))
print('---')
print(classification_report(y_validate.actual, y_validate.rc_cv_pred))

 RC_CV Accuracy: 59.01463%
---
Confusion Matrix
actual               Very Early Response  Early Response  On Time Response  \
rc_cv_pred                                                                   
Early Response                        24              20                12   
Late Response                       1306             815               182   
Very Early Response                47648           15010              4843   
Very Late Response                     0               0                 0   

actual               Late Response  Very Late Response  
rc_cv_pred                                              
Early Response                   9                   3  
Late Response                 2185                 388  
Very Early Response          12205                 300  
Very Late Response               0                 683  
---
                     precision    recall  f1-score   support

     Early Response       0.29      0.00      0.00     15845
      Late Res

### Takeaways

- This model only improves on the baseline by about 1%

## SGD Classifier - Penalty Adjusted to Elastic - Selected Features 

- Lastly we attempted to used an SGD Classifier model

- The version of this model we found to be the best was adjusting the penalty to elastic and using our selected features 

In [29]:
#make the thing
sgd = SGDClassifier(max_iter=1000, tol=1e-3, random_state=123, penalty='elasticnet')
#fit the thing
sgd = sgd.fit(train_scaled, y_train.actual)
#predicitons
y_train['sgd_pred'] = sgd.predict(train_scaled)
y_validate['sgd_pred'] = sgd.predict(validate_scaled)
y_test['sgd_pred'] = sgd.predict(test_scaled)

In [30]:
# calculate accuracy of sgd on train
sgd_accuracy_train = accuracy_score(y_train.actual, y_train.sgd_pred)                                     
print(' SGD Accuracy: {:.5%}'.format(accuracy_score(y_train.actual, y_train.sgd_pred)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(y_train.sgd_pred, y_train.actual))
print('---')
print(classification_report(y_train.actual, y_train.sgd_pred))

 SGD Accuracy: 58.58265%
---
Confusion Matrix
actual               Very Early Response  Early Response  On Time Response  \
sgd_pred                                                                     
Early Response                         7              10                 5   
Late Response                        339             184                60   
On Time Response                      18               9                12   
Very Early Response               113881           36714             11670   
Very Late Response                    35              54                 6   

actual               Late Response  Very Late Response  
sgd_pred                                                
Early Response                   5                   0  
Late Response                  942                  44  
On Time Response                 6                   0  
Very Early Response          32247                 954  
Very Late Response             399                2209  
---
    

In [31]:
# calculate accuracy of sgd on validate
sgd_accuracy_validate = accuracy_score(y_validate.actual, y_validate.sgd_pred)                                     
print(' SGD Accuracy: {:.5%}'.format(accuracy_score(y_validate.actual, y_validate.sgd_pred)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(y_validate.sgd_pred, y_validate.actual))
print('---')
print(classification_report(y_validate.actual, y_validate.sgd_pred))

 SGD Accuracy: 58.63978%
---
Confusion Matrix
actual               Very Early Response  Early Response  On Time Response  \
sgd_pred                                                                     
Early Response                         4               6                 5   
Late Response                        154              83                25   
On Time Response                       9               8                 9   
Very Early Response                48803           15729              4993   
Very Late Response                     8              19                 5   

actual               Late Response  Very Late Response  
sgd_pred                                                
Early Response                   0                   0  
Late Response                  424                  10  
On Time Response                 4                   0  
Very Early Response          13807                 391  
Very Late Response             164                 973  
---
    

### Takeaways

- This model also only improved on the baseline by about 1%

# Evaluation 

- Train Accuracy

In [32]:
print(' Baseline Accuracy: {:.5%}'.format(baseline_accuracy))
print(' Log Accuracy: {:.5%}'.format(log_accuracy_train))
print(' DT Accuracy: {:.5%}'.format(dt_accuracy_train))
print(' KNN Accuracy: {:.5%}'.format(knn_accuracy_train))
print(' RF Accuracy: {:.5%}'.format(rf_accuracy_train))
print(' RC_CV Accuracy: {:.5%}'.format(rc_cv_accuracy_train))
print(' SGD Accuracy: {:.5%}'.format(sgd_accuracy_train))

 Baseline Accuracy: 57.19433%
 Log Accuracy: 59.30184%
 DT Accuracy: 67.96507%
 KNN Accuracy: 60.23923%
 RF Accuracy: 66.55923%
 RC_CV Accuracy: 58.98954%
 SGD Accuracy: 58.58265%


- Validate Accuracy

In [33]:
print(' Baseline Accuracy: {:.5%}'.format(baseline_accuracy))
print(' Log Accuracy: {:.5%}'.format(log_accuracy_validate))
print(' DT Accuracy: {:.5%}'.format(dt_accuracy_validate))
print(' KNN Accuracy: {:.5%}'.format(knn_accuracy_validate))
print(' RF Accuracy: {:.5%}'.format(rf_accuracy_validate))
print(' RC_CV Accuracy: {:.5%}'.format(rc_cv_accuracy_validate))
print(' SGD Accuracy: {:.5%}'.format(sgd_accuracy_validate))

 Baseline Accuracy: 57.19433%
 Log Accuracy: 59.34394%
 DT Accuracy: 68.01467%
 KNN Accuracy: 59.45138%
 RF Accuracy: 66.43934%
 RC_CV Accuracy: 59.01463%
 SGD Accuracy: 58.63978%


### Takeaways

- The decision tree model performed best on unseed validate data, so we will use it on test data

In [34]:
# calculate accuracy of dt on validate
dt_accuracy_test = accuracy_score(y_test.actual, y_test.dt_pred)                                     
print(' DT Accuracy: {:.5%}'.format(accuracy_score(y_test.actual, y_test.dt_pred)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(y_test.dt_pred, y_test.actual))
print('---')
print(classification_report(y_test.actual, y_test.dt_pred))

 DT Accuracy: 67.99512%
---
Confusion Matrix
actual               Very Early Response  Early Response  On Time Response  \
dt_pred                                                                      
Early Response                       453            3515               679   
Late Response                        400             642               170   
Very Early Response                39962            9047              3348   
Very Late Response                     0               0                 0   

actual               Late Response  Very Late Response  
dt_pred                                                 
Early Response                1162                  36  
Late Response                 4224                 189  
Very Early Response           6612                  99  
Very Late Response               2                 821  
---
                     precision    recall  f1-score   support

     Early Response       0.60      0.27      0.37     13204
      Late Respon

In [35]:
print(' Baseline Accuracy: {:.5%}'.format(baseline_accuracy))
print(' DT Accuracy: {:.5%}'.format(dt_accuracy_test))

 Baseline Accuracy: 57.19433%
 DT Accuracy: 67.99512%


### Takeaway

- The final model performs better than baseline by about 10% 

- While this is an improvement there is still room for improvement in future iterations